In [ ]:
from datetime import datetime

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.util as cutil
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import metpy.calc as mpcalc
import metpy.constants as mpconstants
from metpy.units import units
import numpy as np
from pyproj import Proj
from scipy.ndimage import gaussian_filter
import xarray as xr

In [ ]:
date = datetime(2020, 1, 17, 12)

# Choose a dataset...

# ds = xr.open_dataset('https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/GFS/Global_0p5deg_ana/TP')

# UCAR RDA Archive - Global 1.0 deg Data from August 1, 1999 to current
# if datetime(1999, 7, 30, 18) <= date <= datetime(2007, 12, 6, 0):
#     ds = xr.open_dataset('https://tds.gdex.ucar.edu/thredds/dodsC/files/g/d083002/grib1/'
#                          f'{date:%Y}/{date:%Y.%m}/fnl_{date:%Y%m%d_%H}_00.grib1')
# elif date > datetime(2007, 12, 6, 0):
#     ds = xr.open_dataset('https://tds.gdex.ucar.edu/thredds/dodsC/files/g/d083002/grib2/'
#                          f'{date:%Y}/{date:%Y.%m}/fnl_{date:%Y%m%d_%H}_00.grib2')

# UCAR RDA Archive - Global 0.25 deg Data from 2015 to current
ds = xr.open_dataset('https://tds.gdex.ucar.edu/thredds/dodsC/files/g/d083003/'
                       f'{date:%Y}/{date:%Y%m}/gdas1.fnl0p25.{date:%Y%m%d%H}.f00.grib2')

# Local File
#ds = xr.open_dataset(f'groundhogs_day_blizzard/GFS_{date:%Y%m%d}_{date:%H}00.nc')

# Set subset parameters
subset = dict(vertical=slice(100*units.hPa, None),
              time=date,
              lat=slice(60, 10),
              lon=slice(360-160, 360-50))

# Subset data and conform vertical coordinate name to temperature levels
tmpk = ds['Temperature_isobaric'].metpy.sel(subset)
rename_dict = {ds.Relative_humidity_isobaric.metpy.vertical.name: tmpk.metpy.vertical.name}
uwnd = ds['u-component_of_wind_isobaric'].rename(rename_dict).metpy.sel(subset)
vwnd = ds['v-component_of_wind_isobaric'].rename(rename_dict).metpy.sel(subset)
relh = ds['Relative_humidity_isobaric'].rename(rename_dict).metpy.sel(subset)

lon, lat = np.meshgrid(uwnd.lon, uwnd.lat)

lev = uwnd.metpy.vertical

# Set up map projections
mapcrs = ccrs.LambertConformal(central_longitude=-100, central_latitude=35, standard_parallels=(30, 60))
datacrs = ccrs.PlateCarree()

# Transform Coordinates ahead of time
tlatlons = mapcrs.transform_points(ccrs.PlateCarree(), lons, lats)
clons = tlatlons[:,:,0]
clats = tlatlons[:,:,1]

# Set isentropic levels to compute
isentlevs = list(range(285, 331, 3)) * units.kelvin

# Calculate isentropic levels
isent_anal = mpcalc.isentropic_interpolation_as_dataset(isentlevs, tmpk, relh, uwnd, vwnd)

In [ ]:
def plot_isentropic(level):
    print(f'Creating the {level} K Level Maps')

    # Smooth isentropic data for plotting
    isentprs = mpcalc.smooth_n_point(isent_anal.pressure.sel(isentropic_level=level*units.K), 9, 5)
    isenttmp = mpcalc.smooth_n_point(isent_anal.temperature.sel(isentropic_level=level*units.K), 9, 5)
    isentrelh = mpcalc.smooth_n_point(isent_anal.Relative_humidity_isobaric.sel(isentropic_level=level*units.K), 9, 5)
    isentu = mpcalc.smooth_n_point(isent_anal['u-component_of_wind_isobaric'].sel(isentropic_level=level*units.K).metpy.convert_units('kt'), 9, 5)
    isentv = mpcalc.smooth_n_point(isent_anal['v-component_of_wind_isobaric'].sel(isentropic_level=level*units.K).metpy.convert_units('kt'), 9, 5)

    # Use simple scheme for thinning wind barbs on plots
    dlon = uwnd.lon[1] - uwnd.lon[0]
    if dlon == 1:
        barb_spacing = 2
    elif dlon == 0.5:
        barb_spacing = 5
    elif dlon == 0.25:
        barb_spacing = 9
    else:
        barb_spacing = 2
    wind_slice = (slice(None, None, barb_spacing), slice(None, None, barb_spacing))

    # Make the plot
    fig = plt.figure(1, figsize=(17,15))

    ax = plt.subplot(111, projection=mapcrs)
    ax.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
    ax.add_feature(cfeature.STATES.with_scale('50m'))

    cf = ax.contourf(clons, clats, isentrelh, range(70, 101, 1), cmap=plt.cm.Greens, norm=plt.Normalize(70,108))
    plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extendrect=True)

    cs = ax.contour(clons, clats, isentprs, range(50, 1001, 50), colors='black')
    plt.clabel(cs, fmt='%d')

    ax.barbs(lons[wind_slice], lats[wind_slice],
             isentu[wind_slice].values, isentv[wind_slice].values,
             transform=ccrs.PlateCarree())

    plt.title(f'GFS - {level} K Pressure (hPa), Relative Humidity (%), and Wind Barbs', loc='left')
    plt.title(f'Valid Time: {date}', loc='right')

    plt.savefig(f'GFS_Isentropic_{level}K_{date:%Y%m%d_%H}00.png', bbox_inches='tight', dpi=150)
    plt.show()
    plt.close()

In [ ]:
#for level in [294, 297, 300, 303, 312, 315, 324, 330]:
for level in [300]:
    plot_isentropic(level)